In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import copy
from model_grav import *

In [2]:
model.load_state_dict(torch.load(r"grav_model_100_npred_mix2att_lin_concat.pt"), strict = False)    
ema_model.load_state_dict(torch.load(r"grav_ema_model_epoch_100_npred_mix2att_lin_concat.pt"), strict=False)

<All keys matched successfully>

In [ ]:
import matplotlib.gridspec as gridspec
import random

num_rows = 3
num_col = 3

n_samples = num_rows*num_col

samples = sample_epsilon_v2(ema_model, n_samples=num_rows*num_col).cpu().detach()

# --- Plotting ---
fig = plt.figure(figsize=(10, 8))  # Increase width for colorbar
gs = gridspec.GridSpec(num_rows, num_col + 1, width_ratios=[1]*num_col + [0.05], wspace=0.1, hspace=0.2)

# for i in range(num_rows * num_col):
#     plt.subplot(num_rows, num_col, i + 1)
#     plt.imshow(samples[i, 0], cmap='gray')
#     plt.axis('off')

# plt.tight_layout()
# plt.show()

# Plot images
for i in range(n_samples):
    row, col = divmod(i, num_col)
    ax = plt.subplot(gs[row, col])
    im = ax.imshow(samples[i, 0], cmap='gray')
    ax.axis('off')

# Colorbar in the last column of the grid
cbar_ax = plt.subplot(gs[:, -1])
plt.colorbar(im, cax=cbar_ax)

plt.tight_layout()
plt.show()

In [ ]:
from tqdm import tqdm

n_samples = 1000
batch_size = 25
all_samples = []

ema_model.eval()
with torch.no_grad():
    for i in tqdm(range(0, n_samples, batch_size)):
        current_batch = min(batch_size, n_samples - i)
        batch = sample_epsilon_v2(ema_model, n_samples=current_batch)  # should be on GPU
        all_samples.append(batch.cpu())  # move to CPU immediately

# Concatenate all and save as one file
samples = torch.cat(all_samples, dim=0)  # shape [1000, 1, 150, 150]
torch.save(samples, "generated_samples_mix2att_lin.pt")
print("✅ Saved generated samples:", samples.shape)

 88%|████████▊ | 35/40 [1:56:42<16:41, 200.25s/it]  